<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/develop/logic/user_controller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
# Move to repo folder and switch to the correct branch
%cd /content/Cloud_Computing

# List all branches (for verification)
!git branch -a

# Update and switch to develop branch
!git fetch origin
!git checkout develop


/content/Cloud_Computing
* develop
  main
  remotes/origin/HEAD -> origin/main
  remotes/origin/develop
  remotes/origin/main
  remotes/origin/tirgul-Noa-Tal
  remotes/origin/tirgul-Omer-Matan
Already on 'develop'
Your branch is up to date with 'origin/develop'.


In [34]:
import os, sys, pprint

REPO_DIR = "/content/Cloud_Computing"
FIREBASE_DIR = REPO_DIR + "/firebase"

# Check folder exists and print contents
print("firebase/ exists:", os.path.isdir(FIREBASE_DIR))
print("Files inside firebase/:")
pprint.pp(os.listdir(FIREBASE_DIR))

# Add firebase folder to Python path
sys.path.append(FIREBASE_DIR)

# Install importnb to allow importing .ipynb notebooks
!pip -q install importnb

from importnb import Notebook

# Import FireBase.ipynb as module
with Notebook():
    import FireBase as fb


firebase/ exists: True
Files inside firebase/:
['FireBase.ipynb', '__pycache__']


In [35]:
def login(username, password):
    """
    Returns True if user exists AND password matches.
    """
    status, user = fb.get_user(username)
    if status != 200 or user is None:
        return False
    return user.get("password") == password
